In [8]:
import pandas as pd
from pathlib import Path

In [9]:
root_path = Path().cwd().parent / "outputs"

In [10]:
data = pd.read_csv(root_path / "objective-evaluation-results.csv")

In [12]:
data = data.sort_values(by="avg_score", ascending=False)

In [15]:
data

,story_id,title,synopsis,approach,avg_score,coherence,inspiration,narrative_fluency,readability,word_complexity
15,fff66562-f284-11ee-b947-00155d2625d3,Celestial Odyssey,In a world where magic and technology intertwi...,proposed,6.814534,7.197239,6.340504,6.855827,7.219188,6.459911
11,61b6774a-f487-11ee-8f74-182649966cd4,The Chronicles of Zephyr,In a world where magic and technology intertwi...,proposed,6.762332,7.071141,6.244810,6.821667,7.055688,6.618354
17,6a8b12bd-03d2-11ef-9093-182649966cd4,The Celestial Chronicles,In a world where magic and technology intertwi...,proposed,6.720377,7.117082,6.252861,6.662638,7.086998,6.482306
12,4aa911cb-f78a-11ee-979e-182649966cd4,Echoes of the Celestial Realm,In a world where ancient magic and advanced te...,proposed,6.692180,7.095310,6.142679,6.735502,7.113932,6.373476
9,c185bc6f-f3dd-11ee-9cd1-182649966cd4,Celestial Odyssey,"In a distant future, Zara Quark, a skilled pil...",proposed,6.667909,6.915770,6.223764,6.615221,7.089462,6.495326
10,ea721ae9-033c-11ef-a17c-182649966cd4,Chronicles of the Celestial Realm,In a world where magic and technology intertwi...,proposed,6.611573,6.866879,6.174120,6.537452,7.050340,6.429077
14,3bd1b7aa-051b-11ef-89d5-182649966cd4,Echoes of the Celestial Realm,In a world where magic and technology intertwi...,proposed,6.606084,7.003498,6.161393,6.488124,6.992972,6.384434
16,3859d8cb-f319-11ee-b719-182649966cd4,Echoes of the Celestial Realm,"In a world where magic and technology coexist,...",proposed,6.579677,6.794315,6.142403,6.524602,7.122895,6.314170
13,790a18bb-f56e-11ee-a333-182649966cd4,Chronicles of the Celestial Realm,In a world where advanced technology and ancie...,proposed,6.561804,6.785779,6.162755,6.549899,7.013471,6.297115
0,d979223f-f4e2-11ee-b819-182649966cd4,The Chronicles of Zephyr,In a world where magic and technology intertwi...,baseline,6.531065,6.827718,6.017821,6.486688,6.955633,6.367465


In [22]:
# get max score for each "avg_score", "coherence", "inspiration", "narrative_fluency", "readability", "word_complexity" across all stories
max_scores = {
    "avg_score": data["avg_score"].max(),
    "coherence": data["coherence"].max(),
    "inspiration": data["inspiration"].max(),
    "narrative_fluency": data["narrative_fluency"].max(),
    "readability": data["readability"].max(),
    "word_complexity": data["word_complexity"].max()
}

In [28]:
# Convert table to LaTeX table row
def to_latex_row(row):
    def map_approach(approach: str):
        return approach == "baseline" and "Baseline" or "DCP/P"
    # return f"{row['story_id'][:8]} & {row['title']} & {map_approach(row['approach'])} & {row['avg_score']:.2f} & {row['coherence']:.2f} & {row['inspiration']:.2f} & {row['narrative_fluency']:.2f} & {row['readability']:.2f} & {row['word_complexity']:.2f} \\\\"
    # Bold the best score using \textbf{}
    story_id = row["story_id"][:8]
    title = row["title"]
    approach = map_approach(row["approach"])
    avg_score = row["avg_score"] == max_scores["avg_score"] and f"\\textbf{{{row['avg_score']:.2f}}}" or f"{row['avg_score']:.2f}"
    coherence = row["coherence"] == max_scores["coherence"] and f"\\textbf{{{row['coherence']:.2f}}}" or f"{row['coherence']:.2f}"
    inspiration = row["inspiration"] == max_scores["inspiration"] and f"\\textbf{{{row['inspiration']:.2f}}}" or f"{row['inspiration']:.2f}"
    narrative_fluency = row["narrative_fluency"] == max_scores["narrative_fluency"] and f"\\textbf{{{row['narrative_fluency']:.2f}}}" or f"{row['narrative_fluency']:.2f}"
    readability = row["readability"] == max_scores["readability"] and f"\\textbf{{{row['readability']:.2f}}}" or f"{row['readability']:.2f}"
    word_complexity = row["word_complexity"] == max_scores["word_complexity"] and f"\\textbf{{{row['word_complexity']:.2f}}}" or f"{row['word_complexity']:.2f}"
    return f"{story_id} & {approach} & {avg_score} & {coherence} & {inspiration} & {narrative_fluency} & {readability} & {word_complexity} \\\\"

In [29]:
for _, row in data.iterrows():
    print(to_latex_row(row))

fff66562 & DCP/P & \textbf{6.81} & \textbf{7.20} & \textbf{6.34} & \textbf{6.86} & \textbf{7.22} & 6.46 \\
61b6774a & DCP/P & 6.76 & 7.07 & 6.24 & 6.82 & 7.06 & \textbf{6.62} \\
6a8b12bd & DCP/P & 6.72 & 7.12 & 6.25 & 6.66 & 7.09 & 6.48 \\
4aa911cb & DCP/P & 6.69 & 7.10 & 6.14 & 6.74 & 7.11 & 6.37 \\
c185bc6f & DCP/P & 6.67 & 6.92 & 6.22 & 6.62 & 7.09 & 6.50 \\
ea721ae9 & DCP/P & 6.61 & 6.87 & 6.17 & 6.54 & 7.05 & 6.43 \\
3bd1b7aa & DCP/P & 6.61 & 7.00 & 6.16 & 6.49 & 6.99 & 6.38 \\
3859d8cb & DCP/P & 6.58 & 6.79 & 6.14 & 6.52 & 7.12 & 6.31 \\
790a18bb & DCP/P & 6.56 & 6.79 & 6.16 & 6.55 & 7.01 & 6.30 \\
d979223f & Baseline & 6.53 & 6.83 & 6.02 & 6.49 & 6.96 & 6.37 \\
8f896f76 & Baseline & 6.40 & 6.76 & 5.91 & 6.25 & 7.00 & 6.10 \\
0f83a38e & Baseline & 6.39 & 6.61 & 6.04 & 6.21 & 6.95 & 6.15 \\
5f2decaf & Baseline & 6.35 & 6.55 & 5.80 & 6.26 & 7.01 & 6.14 \\
c5b48f11 & Baseline & 6.30 & 6.44 & 5.86 & 6.22 & 6.78 & 6.21 \\
681f53ec & Baseline & 6.27 & 6.42 & 5.67 & 6.23 & 6.87 & 6.19 \